<a href="https://colab.research.google.com/github/pacomm5/PLANTILLAS2/blob/main/COHORT3_MIO_Carroceria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AQUI VOY A HACER EL DE CARROCERÍA PARA MARKETING ESTE ES EL NUEVO. 
este es el nuevo despues de corregir

In [4]:
import pandas as pd
# me quedo aquí tengo que volver a hacerlo a ver si me aclaro. 
# entiendo que al quitar los duplicados de las orden corta2, también me estoy cargando el bastidor y me va a salir mal. 

In [5]:
xlsx = pd.ExcelFile("/content/drive/MyDrive/Colab Notebooks/U239_IMPORTADO.xlsx") # ESTE EL NOMBRE DE LA RUTA. 
df = pd.read_excel(xlsx, "U239_IMPORTADO" ) # ESTE EL NOMBRE DE LA PESTAÑA. 
# he descargado el archivo en google drive para no tener que estar cargarlo cada vez que lo utilice. 

In [33]:
# for data manipulation and analysis
import pandas as pd
import numpy as np

# for plotting
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
sns.set_style('darkgrid')

# Silhouette analysis
from sklearn.metrics import silhouette_score

# To perform KMeans clustering 
from sklearn.cluster import KMeans

# for scaling
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [34]:
pd.options.display.max_columns = 0 # con esto n olimitamos que se vean todas las columnas a lo ancho

In [35]:
df.columns

Index(['Tall', 'Referencia', 'Fecha_Fact', 'Fecha_Apertura', 'Cta_Cargo',
       'Nombre_Cliente', 'Cta_Titular', 'Nombre_Titular', 'Recamb',
       'Recamb_Pvp', 'Tiemp_Fact', 'Total_Mo', 'Modelo', 'Recep', 'Matricula',
       'Bastidor', 'Tipo_Or', 'Or_Unica', 'MES', 'AÑO', 'ESTANCIA',
       'Orden_Corta', 'Fact_total', 'Orden_Corta2'],
      dtype='object')

In [36]:
# voy a cambiar el nombre de algunas columnas. 
df = df.rename(columns={"Fecha Fa":"Fecha_Fact", "Fec.aper":"Fecha_Apertura", "Cta.car":"Cta_Cargo","Nombre cliente":"Nombre_Cliente", "Cta.tit":"Cta_Titular", "Nombre titular":"Nombre_Titular", 
                   "Recamb.":"Recamb", "RECAMBIOS PVP":"Recamb_Pvp", "Tiem.fac":"Tiemp_Fact", "Total.MO":"Total_Mo", "Tipo O.R.":"Tipo_Or", "OR UNICA": "Or_Unica", "ORDEN CORTA":"Orden_Corta" })


In [37]:
print("la fecha va desde {} a {}".format(df.Fecha_Fact.min(), df.Fecha_Fact.max()))

la fecha va desde 2015-01-02 00:00:00 a 2022-12-30 00:00:00


In [38]:
# lo primero que voy a hacer es cambiar de tipo de datos. 
df["Tall"] = df["Tall"].apply(str)
df["Referencia"] = df["Referencia"].apply(str)
df["Cta_Cargo"] = df["Cta_Cargo"].apply(str)
df["Or_Unica"] = df["Or_Unica"].apply(str)
df["Orden_Corta"] = df["Orden_Corta"].apply(str)
df["Cta_Cargo"] = df["Cta_Cargo"].apply(str)
df["Nombre_Cliente"] = df["Nombre_Cliente"].apply(str)
df["Cta_Titular"] = df["Cta_Titular"].apply(str)
df["Nombre_Titular"] = df["Nombre_Titular"].apply(str)
df["Modelo"] = df["Modelo"].apply(str)
df["Recep"] = df["Recep"].apply(str)
df["Matricula"] = df["Matricula"].apply(str)
df["Bastidor"] = df["Bastidor"].apply(str)
df["Tipo_Or"] = df["Tipo_Or"].apply(str)


In [39]:
df["Fact_total"] = df["Total_Mo"] + df["Recamb"]

In [59]:
df.tail()

,Tall,Referencia,Fecha_Fact,Fecha_Apertura,Cta_Cargo,Nombre_Cliente,Cta_Titular,Nombre_Titular,Recamb,Recamb_Pvp,Tiemp_Fact,Total_Mo,Modelo,Recep,Matricula,Bastidor,Tipo_Or,Or_Unica,MES,AÑO,ESTANCIA,Orden_Corta,Fact_total,Orden_Corta2
307972,1,14213091,NaT,2022-12-29,137273,TERAN BARRIOBERO EUGENI,137273,TERAN BARRIOBERO EUGENI,0.00,0.00,0.0,0.00,A5,241,5977HCN,WAUZZZ8T3BA067279,2A O.R.CLIENT MEC AUDI-CO,nan,1,1900,-44924,1421309,0.00,1421309
307973,3,14213621,2022-12-30,2022-12-29,136805,MORALEDA MORALEDA SATUR,136805,MORALEDA MORALEDA SATUR,213.70,427.40,3.2,257.86,AUDI Q3 DE,294,7816JMZ,WAUZZZ8U2GR095043,2A O.R.CLIENT MEC AUDI-CO,nan,12,2022,1,1421362,471.56,1421362
307974,3,14214141,2022-12-29,2022-12-29,156237,SONIA MARTINEZ ROBLES,156237,SONIA MARTINEZ ROBLES,0.00,0.00,0.0,0.00,AUDI Q3 BL,63,7114KNX,WAUZZZ8U4JR099313,2A O.R.CLIENT MEC AUDI-CO,nan,12,2022,0,1421414,0.00,1421414
307975,1,14214201,2022-12-29,2022-12-29,156237,SONIA MARTINEZ ROBLES,156237,SONIA MARTINEZ ROBLES,0.00,41.32,0.0,0.00,AUDI Q3 BL,63,7114KNX,WAUZZZ8U4JR099313,2A O.R.CLIENT MEC AUDI-CO,nan,12,2022,0,1421420,0.00,1421420
307978,6,14215461,2022-12-30,2022-12-30,44721,Jose Maria Prieto Jimen,44721,Jose Maria Prieto Jimen,181.05,394.05,0.8,62.56,AUDI A6 3.,41,2860HVF,WAUZZZ4G7EN085953,2A O.R.CLIENT MEC AUDI-CO,nan,12,2022,0,1421546,243.61,1421546


In [40]:
# de esta manera lo que hago es quitar el último digito de la variable referencia. 
# lo he hecho para probar como se hace. podría haberlo metido en otra variable y luego meterla en el dataframe para no eliminar información. 
df["Orden_Corta2"] = df.apply(lambda x: x["Referencia"][:-1], axis=1)

In [41]:
# voy a quitar las internas. 
df = df[df["Tipo_Or"].str.startswith("5")!=True]
# voy a quitar las internas. 
df = df[df["Tipo_Or"].str.startswith("6")!=True]

In [42]:
df_carroceria = df[df["Tipo_Or"].str.startswith("7")==True]

In [43]:
# AQUI ESTOY COMPROBANDO QUE SOLO ME HAYA QUEDADO CON CARROCERÍA. 
df_carroceria.Tipo_Or.value_counts(normalize=True) # por numero de lineas, debajo están los pesos. 

7A PRESUPUESTO CH AUDI       0.574434
7V PRESUPUESTO CH VW         0.381218
7LA PRES  AU CH  LONG DRI    0.025749
7LV PRES  VW CH  LONG DRI    0.013439
7L PRESUPUESTO CH LCV        0.004408
7LL PRES  LCV  CH  LONG D    0.000753
Name: Tipo_Or, dtype: float64

In [48]:
# antes de nada lo que voy a hacer es ver los valores null
df_carroceria.isnull().sum()

Tall                 0
Referencia           0
Fecha_Fact        1640
Fecha_Apertura       0
Cta_Cargo            0
Nombre_Cliente       0
Cta_Titular          0
Nombre_Titular       0
Recamb               0
Recamb_Pvp           0
Tiemp_Fact           0
Total_Mo             0
Modelo               0
Recep                0
Matricula            0
Bastidor             0
Tipo_Or              0
Or_Unica             0
MES                  0
AÑO                  0
ESTANCIA             0
Orden_Corta          0
Fact_total           0
Orden_Corta2         0
dtype: int64

In [61]:
df_carroceria = df[~df.Fecha_Fact.isnull()] # los quito

In [62]:
df_carroceria.isnull().sum() # tengo algunos pero no voy  aseguir quitando. 

Tall              0
Referencia        0
Fecha_Fact        0
Fecha_Apertura    0
Cta_Cargo         0
Nombre_Cliente    0
Cta_Titular       0
Nombre_Titular    0
Recamb            0
Recamb_Pvp        0
Tiemp_Fact        5
Total_Mo          5
Modelo            0
Recep             0
Matricula         0
Bastidor          0
Tipo_Or           0
Or_Unica          0
MES               0
AÑO               0
ESTANCIA          0
Orden_Corta       0
Fact_total        5
Orden_Corta2      0
dtype: int64

In [51]:
# vuelvo a comprobar si hay duplicados 
num_duplicados = df_carroceria["Orden_Corta2"].duplicated().sum() # con esto sabemos cuantos duplicados hay en la columna. 

In [63]:
num_duplicados # y como vemos hay muchos duplicados. 
# por lo tanto lo primero que vamos a hacer es quitar los duplicados para quitar el número de visitas. 

0

In [53]:
# primero hago una copia del df_carrocería para no perderlo. 
df_carroceria_sinduplicados_orcorta2 = df_carroceria.copy()

In [58]:
df_carroceria_sinduplicados_orcorta2.tail()

,Tall,Referencia,Fecha_Fact,Fecha_Apertura,Cta_Cargo,Nombre_Cliente,Cta_Titular,Nombre_Titular,Recamb,Recamb_Pvp,Tiemp_Fact,Total_Mo,Modelo,Recep,Matricula,Bastidor,Tipo_Or,Or_Unica,MES,AÑO,ESTANCIA,Orden_Corta,Fact_total,Orden_Corta2
34,2,7747121,2015-01-02,2014-12-30,104188,PARRA TOLEDO MATILDE,104188,PARRA TOLEDO MATILDE,0.00,0.00,0.50,36.50,POLO 1.9 S,85,4873DCS,WVWZZZ9NZ5Y069802,2V O.R.CLIENT MEC VW,nan,1,2015,3,774712,36.50,774712
33,2,7747031,2015-01-02,2014-12-29,126338,CORDERO HERNANDEZ M ISA,126338,CORDERO HERNANDEZ M ISA,469.17,991.90,2.05,139.70,TOUAREG,9,8394FLK,WVGZZZ7LZ7D065443,2V O.R.CLIENT MEC VW,nan,1,2015,4,774703,608.87,774703
32,2,7746791,2015-01-02,2014-12-30,112379,CARMONA PARRA VANESA,112379,CARMONA PARRA VANESA,9.68,22.98,0.70,35.77,POLO ADVAN,50,7991HGN,WVWZZZ6RZCY070767,2V O.R.CLIENT MEC VW,nan,1,2015,3,774679,45.45,774679
30,6,7746161,2015-01-02,2014-12-29,120718,OSKAR SCHUNCK ESPAñA,120718,OSKAR SCHUNCK ESPAñA,207.04,465.84,2.10,142.10,AUDI A6 2.,56,5248HML,WAUZZZ4G1CN153418,2A O.R.CLIENT MEC AUDI,nan,1,2015,4,774616,349.14,774616
24,2,7741801,2015-01-02,2015-01-02,97987,MIRA GOMEZ PALOMA,97987,MIRA GOMEZ PALOMA,197.25,429.30,1.60,116.80,TOURAN EDI,9,4533GKB,WVGZZZ1TZ9W031579,2V O.R.CLIENT MEC VW,nan,1,2015,0,774180,314.05,774180


In [55]:
# ahora lo que hago es ordenarlos por fecha de Facturación en orden descedente y me quedo con la primera, que es la fecha más reciente. 
df_carroceria_sinduplicados_orcorta2 = df_carroceria_sinduplicados_orcorta2.sort_values(by="Fecha_Fact", ascending=False).drop_duplicates(
    subset=["Orden_Corta2"], keep= "first")

In [56]:
# voy a comprobar si tengo duplicados después de hacerlo.
num_duplicados = df_carroceria_sinduplicados_orcorta2["Orden_Corta2"].duplicated().sum()
num_duplicados
# y vemos que no tenemos ningún duplicado. 

0

In [57]:
# voy a comprobar si tengo duplicados después de hacerlo.
num_duplicados2 = df_carroceria_sinduplicados_orcorta2["Bastidor"].duplicated().sum()
num_duplicados
# y vemos que no tenemos ningún duplicado. 

0

In [30]:
# ahora sí puedo calcular el numero de visitas. 
visitas = pd.DataFrame(df_carroceria_sinduplicados.groupby("Bastidor")["Orden_Corta2"].nunique().reset_index())
visitas.columns = ["vBastidor", "visitas"]
visitas.head()

,vBastidor,visitas
0,WAUZZZ4L97D07664,5
1,WAUZZZ8E55A50910,1
2,WAUZZZ8P07A02124,2
3,WAUZZZ8P1CA04845,1
4,WAUZZZ8T4EA06800,6


In [ ]:
visitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51759 entries, 0 to 51758
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   vBastidor  51759 non-null  object
 1   visitas    51759 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 808.9+ KB


In [ ]:
ultima_visita = pd.DataFrame(df_carroceria.groupby("Bastidor")["Fecha_Fact"].max().reset_index())
ultima_visita.columns = ["uBastidor", "Ulti_visita"]
ultima_visita.head()


,uBastidor,Ulti_visita
0,WAUZZZ4L97D07664,2020-10-09
1,WAUZZZ8E55A50910,2017-12-20
2,WAUZZZ8P07A02124,2021-09-06
3,WAUZZZ8P1CA04845,2016-04-15
4,WAUZZZ8T4EA06800,2020-03-16


In [ ]:
num_duplicados = ultima_visita["uBastidor"].duplicated().sum() # con esto sabemos cuantos duplicados hay en la columna.
num_duplicados

0

In [ ]:
ultima_visita.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51759 entries, 0 to 51758
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   uBastidor    51759 non-null  object        
 1   Ulti_visita  51759 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 808.9+ KB


In [ ]:
df_carroceria_marketing = pd.concat([visitas, ultima_visita], axis=1)
df_carroceria_marketing.drop([ "uBastidor"], axis=1, inplace=True)
df_carroceria_marketing.head()

,vBastidor,visitas,Ulti_visita
0,WAUZZZ4L97D07664,5,2020-10-09
1,WAUZZZ8E55A50910,1,2017-12-20
2,WAUZZZ8P07A02124,2,2021-09-06
3,WAUZZZ8P1CA04845,1,2016-04-15
4,WAUZZZ8T4EA06800,6,2020-03-16


In [ ]:
num_duplicados = df_carroceria_marketing["vBastidor"].duplicated().sum() # con esto sabemos cuantos duplicados hay en la columna.
num_duplicados

0

In [ ]:
df_carroceria_marketing.to_excel("/content/drive/MyDrive/Colab Notebooks/Carroceria_unicos.xlsx", index=False)